In [1]:
%%bash
lein uberjar

Retrieving clj-time/clj-time/0.14.0/clj-time-0.14.0.jar from clojars
Compiling ppdsp.classifier.base
Compiling ppdsp.classifier.inspectablearf
Compiling ppdsp.classifier.moa-classifier
Compiling ppdsp.classifier.random
Compiling ppdsp.core
Compiling ppdsp.dataset.base
Compiling ppdsp.dataset.csv-dataset
Compiling ppdsp.dataset.moa
Compiling ppdsp.dataset.save-csv
Compiling ppdsp.masking.attack-data
Compiling ppdsp.masking.base
Compiling ppdsp.masking.evaluation
Compiling ppdsp.masking.optimize
Compiling ppdsp.masking.projection
Compiling ppdsp.masking.single-stage-cumulative-attack
Compiling ppdsp.masking.two-stage-cumulative-attack
Compiling ppdsp.masking.two-stage-independent-attack
Compiling ppdsp.masking.utils
Compiling ppdsp.scratchpad
Compiling ppdsp.training
Compiling ppdsp.trees.base
Compiling ppdsp.trees.graph-tree
Compiling ppdsp.trees.parse-model
Compiling ppdsp.utils.complex-numbers
Compiling ppdsp.utils.matrices
Compiling ppdsp.utils.moving-average
Compiling ppdsp.utils.ra

In [7]:
 %classpath add jar ../target/jvm/uberjar/ppdsp-0.1.0-SNAPSHOT-standalone.jar
(clojure.lang.Compiler/loadFile "jupyter_helpers.clj")
(require '[clojure.string :as string]
         '[clojure.data.csv :as csv]
         ;;'[incanter core io charts stats]
         ;'[incanter.core :only [to-matrix]]
        ;; '[incanter.io :as inio]
         ;;'[incanter.charts :as incharts]
        ;; '[incanter.stats :as instats ]
          ;[clojure.math.numeric-tower :refer [sqrt expt]]
         '[clojure.java.io :as io]
         '[clojure.pprint :refer [pprint print-table]]
         '[jupyter-helpers :refer [save-data load-data display-table ;testing
                                  ;; display-masking-error-plots  
                                  ;; attack-strategy-comparison-plots 
                                   ;;noise-accuracy-plot accuracy-privacy-tradeoff-comparison
                                  ;; display-html accuracy-privacy-tradeoff-legend
                                   ;accuracy-privacy-tradeoff attack-strategy-comparison
                                   round-known-record-counts
                                   ;;added by Waruni
                                    read-data run-masking-experiments-cycles     accuracy-updating-cycles   
                                     save-data-append-newline  ;;accuracy-for-flat-areas 
                                   attack-per-cycle
                                   ]]
         '[ppdsp.classifier.moa-classifier :refer [hoeffding-tree ]]    ;;changed
         '[ppdsp.dataset.base :refer [dataset-feature-count dataset-record-count dataset-class-count get-schema]]
         '[ppdsp.dataset.csv-dataset :refer [read-csv-dataset]]
         '[ppdsp.masking.evaluation :refer [flatten-masking-experiment-recoveries
                                           unknown-record-relative-position
                                           add-combined-result
                                           get-cumulative-noise-sigma  calculate-avarage-information-loss 
                                            get-attack-count
                                               ]]
         '[ppdsp.utils :refer [map-vals mean]])
(use '(incanter core optimize charts))


null

## Experiment Configuration

In [8]:
(def dataset-label "arem")

#'beaker_clojure_shell_df8f0f9f-ea4f-41b3-bcdf-0adffcb522c2/dataset-label

## Run Experiments

In [14]:
;;;;;;;;;;;;;;;;;;;Read Data from existing csv files 
;(def privacy-breach (to-matrix (load-data (str "workspace/" dataset-label "/final-breach-probabilities.csv"))))
(def accuracy (to-matrix (load-data (str "workspace/" dataset-label "/masked-AEL.csv"))))


;;;;;;;;;;;;;;;;;;
;(def data (to-matrix (get-dataset :chwirut)))
;(println (get-dataset :chwirut))
;(def x (sel data :cols 1))
;(def y (sel data :cols 0))
;(def plot (scatter-plot x y :legend true))

;(defn f [theta x]
 ; (let [[b1 b2 b3] theta]
   ; (let [[b c] theta]
      ;(div (exp (mult (minus b1) x)) (plus b2 (mult b3 x)))
      ;(exp (div (minus x b1) (mult x b2) ))
    ; (exp (div (mmult 2 (sq (minus x b1))) (mult (* b2 b2) 2)))
   ;)
;)
;(println (nrow x))
;(def start1 [1 2 0.02])  ; 
;(add-lines plot x (f start1 x))
;(def nlm1 (non-linear-model f y x start1))
;(println (:coefs nlm1))
;(add-lines plot x (:fitted nlm1))
;(println nlm1)
;(view plot)

clojure.lang.ExceptionInfo:  Dimension out of range

In [9]:
;;Accuracy per sample for logistic noise
(def accuracy-cycles-outputfile-logistic (str "workspace/" dataset-label "/logistic-accuracy-per-cycle.edn"))
(def details-outputfile-logistic (str "workspace/" dataset-label "/logistic-accuracy-details.edn"))
(def full-cycle 1000)
(def access-result-list-logistic (nth results-cumulative-logistic 0)) ;;nth is used to access elements of lists
 (def acc-after-masking-logistic (-> access-result-list-logistic :accuracy :hdt :raw-results )) ;;getting only accuracy results
 

       ;;(accuracy-updating-cycles acc-after-masking-logistic accuracy-cycles-outputfile-logistic record-count full-cycle details-outputfile-logistic)

#'beaker_clojure_shell_e4557b0b-9ecd-4d69-9c9e-4695b7aba92e/acc-after-masking-logistic

In [16]:
;;Average Accuracy for the attacked cycle period (Starting flat period of the logistic cycle)
(def accuracy-details-file (load-data (str "workspace/" dataset-label "/logistic-accuracy-details.edn"))) ;;reading accuracy details from previously written file
(def accuracy-for-attcked-period-file (str "workspace/" dataset-label "/logistic-accuracy-attacked-period.csv")) ;; write file of relavant accuracy details
(def accuracy-details  (-> accuracy-details-file :sample-accuracy ))  
(def full-cycle 1000)
(def flat-record-length (:flat-record-length logistic-cumulative-noise-configuration))
       (accuracy-for-flat-areas accuracy-details full-cycle flat-record-length record-count accuracy-for-attcked-period-file)


java.io.FileNotFoundException:  workspace/arem/logistic-accuracy-details.edn (No such file or directory)

In [17]:
(def get-breach-probabilty (load-data (str "workspace/" dataset-label "/logistic-attacks-per-cycle.edn"))) ;;get the final breach probability written in the file
(save-data-append-newline (str "workspace/" dataset-label "/logistic-breach-probability.csv") get-breach-probabilty) ;;append the value to another file for the operations after 100 runs

java.io.FileNotFoundException:  workspace/arem/logistic-attacks-per-cycle.edn (No such file or directory)

## Comparison of Attack Strategies

In [18]:
(doseq [epsilon epsilons cycles cycle-sizes]
    (display-html (str "<h3>Logistic Cumulative Noise With Cycles" "</h3>" "<h3>Epsilon = " epsilon "</h3>" "<h3>Cycle Size = " (* cycles 2 )"</h3>" ))
    (.display (attack-strategy-comparison-plots flat-results-cumulative-logistic :all epsilon
                                              :plot-width 400
                                              :plot-height 400
                                              :show-legend? true))
    (display-html (str "<h4>Breakdown by noise amount</h4>"))
    (.display (attack-strategy-comparison-plots flat-results-cumulative-logistic :cumulative-noise-sigma epsilon
                                                :plot-width 400
                                                :plot-height 400
                                                :show-legend? false)))

java.lang.RuntimeException:  Unable to resolve symbol

In [19]:
(let [comparison (attack-strategy-comparison flat-results-cumulative-cycles epsilons
                                  :known-record-count (apply max (:known-record-counts base-configuration)))]
    (save-data (str "workspace/" dataset-label "/cumulative-cycles-attack-strategies-comparison.edn") comparison)
    (display-table comparison))

java.lang.RuntimeException:  Unable to resolve symbol

In [20]:
(let [comparison (attack-strategy-comparison flat-results-cumulative-logistic epsilons
                                  :known-record-count (apply max (:known-record-counts base-configuration)))]
    (save-data (str "workspace/" dataset-label "/logistic-attack-strategies-comparison.edn") comparison)
    (display-table comparison))


java.lang.RuntimeException:  Unable to resolve symbol

In [21]:
;;(def best-cumulative-attack-strategy :a-rpcn-1)
(def best-cumulative-cycles-attack-strategy :a-rpcn-1)
(def best-logistic-attack-strategy :a-rpcn-1)

#'beaker_clojure_shell_23e4d944-a124-4fb2-92e2-c6028b05c2dd/best-logistic-attack-strategy

##Linear Cumulative vs.Logistic Cumulative Noise

In [22]:
(display-html (accuracy-privacy-tradeoff-legend cumulative-noise-sigmas))
nil

java.lang.RuntimeException:  Unable to resolve symbol

In [23]:
(doseq [epsilon epsilons]
    (.display (accuracy-privacy-tradeoff-comparison  results-cumulative-cycles results-cumulative-logistic ;;results-cumulative
                                                    :arf  best-cumulative-cycles-attack-strategy best-logistic-attack-strategy epsilon  ;;best-cumulative-attack-strategy
                                                    :plot-width 400
                                                    :plot-height 400)))

java.lang.RuntimeException:  Unable to resolve symbol

### Comparison of Square Distance From Origin

Performance is the sum of the squares of: (1) Prob. of e-privacy breach and (2) classification error.

In [24]:
(let [comparison (accuracy-privacy-tradeoff  results-cumulative-cycles results-cumulative-logistic ;;results-cumulative
                               :arf  best-cumulative-cycles-attack-strategy best-logistic-attack-strategy epsilons ;;best-cumulative-attack-strategy
                               :square-distance? true
                               :row-per-noise-level? true
                               :known-record-count (apply max (:known-record-counts base-configuration))
                                           )]
    (save-data (str "workspace/" dataset-label "/mask-comparison.edn") comparison)
    (display-table comparison))

java.lang.RuntimeException:  Unable to resolve symbol

## Effect of Cumulative Noise on Accuracy over Time

In [25]:
(noise-accuracy-plot  results-cumulative-cycles results-cumulative-logistic :arf ;;results-cumulative
                     :init-width 800
                     :init-height 400)

java.lang.RuntimeException:  Unable to resolve symbol

## Effect of Cumulative Noise on Privacy over Time

In [26]:
(display-masking-error-plots (->> flat-results-cumulative-cycles
                                  (filter #(= best-cumulative-cycles-attack-strategy (:strategy %)))
                                  (filter #(= (apply max cumulative-noise-sigmas) (:cumulative-noise-sigma %)))
                                  (filter #(= (apply max (:known-record-counts base-configuration)) (:known-record-count %))))
                             :cumulative-noise-sigma :known-record-count :strategy
                             :plot-width 500
                             :plot-height 500)

java.lang.RuntimeException:  Unable to resolve symbol

In [27]:
(display-masking-error-plots (->> flat-results-cumulative-logistic
                                  (filter #(= best-logistic-attack-strategy (:strategy %)))
                                  (filter #(= (apply max cumulative-noise-sigmas) (:cumulative-noise-sigma %)))
                                  (filter #(= (apply max (:known-record-counts base-configuration)) (:known-record-count %))))
                             :cumulative-noise-sigma :known-record-count :strategy
                             :plot-width 500
                             :plot-height 500)

java.lang.RuntimeException:  Unable to resolve symbol